<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [ ]:
!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [1]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) > 0, "Not enough GPU hardware devices available"
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

2024-04-10 10:51:57.275472: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 10:51:58.704779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 10:51:58.705045: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-

In [3]:
import sys
sys.path.append("..")

In [4]:
%load_ext autoreload
%autoreload 2

### Reading Data Files:

In [5]:
from mtsa import calculate_aucroc

In [6]:
from mtsa import files_train_test_split
#path_input_1 = '/data/MIMII/fan/id_00/'
X_train, X_test, y_train, y_test = files_train_test_split(path_input_1)
if(len(y_train) == 0): 
    X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
y_train

array([1., 1., 1., 1., 1.])

### MFFCMix Model:

In [6]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [7]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.64

### Hitachi Model:

In [5]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

ValueError: Memory growth cannot differ between GPU devices

In [8]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

14/14 [==============================] - 0s 1ms/step


0.24

### RANSynCoders

In [7]:
from mtsa import RANSynCoders
model_ransyncorders = RANSynCoders(mono=False)
model_ransyncorders.fit(X_train, y_train)

2024-04-10 10:52:10.751283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 10:52:10.751541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-10 10:52:10.751757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf


Start of frequency pre-train epoch 0
pre-reconstruction_loss: 0.04073125
Start of frequency pre-train epoch 1
pre-reconstruction_loss: 0.004788098
Start of frequency pre-train epoch 2
pre-reconstruction_loss: 0.0051332195
Start of frequency pre-train epoch 3
pre-reconstruction_loss: 0.0049807327
Start of frequency pre-train epoch 4
found common oscillations at period(s) = [116.07661056297155, 85.89220528237061, 250.86233929131387, 443.7049362174154, 125.4115065057219]

Start of sine representation pre-train epoch 0
sine_loss: 0.041041672
Start of sine representation pre-train epoch 1
sine_loss: 0.005608797
Start of sine representation pre-train epoch 2
sine_loss: 0.005632456
Start of sine representation pre-train epoch 3
sine_loss: 0.005616599
Start of sine representation pre-train epoch 4
sine_loss: 0.005676355
Start of epoch 0
sine_loss: 0.0056563807 upper_bound_loss: 0.0017412978 lower_bound_loss: 0.0017600069
Start of epoch 1
sine_loss: 0.00551991 upper_bound_loss: 0.0017007492 lo

Pipeline(steps=[('wav2array', Wav2Array(mono=False, sampling_rate=16000)),
                ('preprocessing_data', PreprocessingData()),
                ('min_max_scaler',
                 MinMaxScalerNormalization(training_mode=False)),
                ('final_model',
                 <mtsa.models.ransyncorders.RANSynCodersBase object at 0x7fe43ce88a30>)])

In [8]:
auc = calculate_aucroc(model_ransyncorders, X_test, y_test)
auc

2024-04-10 11:05:50.114336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [160000,8]
	 [[{{node Placeholder/_1}}]]
2024-04-10 11:05:51.693957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [160000,8]
	 [[{{node Placeholder/_1}}]]
2024-04-10 11:05:52.926588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape 

0.76